In [5]:
import pickle
import utils
import json

In [2]:
aux_path='auxiliary_data'
all_docs_file = '%s/docs.conll' % aux_path # 
more_docs_file='%s/new.conll'  % aux_path #
relevant_docs_file='%s/final_docs.p'  % aux_path # 

input_path='../data/input'
out_pickle = '%s/text/docs.p' % input_path

### 1) Get and store texts to disk

In [3]:
# Load all necessary components
all_docs_data=utils.load_conll_data(all_docs_file)

more_docs_data=utils.load_conll_data(more_docs_file)

with open(relevant_docs_file, 'rb') as f:
    relevant_incdocs=pickle.load(f)
    


In [4]:
# Merge initial and added documents
for more_doc, more_doc_data in more_docs_data.items():
    if more_doc not in all_docs_data.keys():
        all_docs_data[more_doc]=more_doc_data

In [5]:
# Combine document IDs from all incidents
relevant_docs=set()
for inc, docs in relevant_incdocs.items():
    relevant_docs |= set(docs)
    
print(len(relevant_docs))

456


In [6]:
# Keep only the data for the relevant documents
docs_data={}
for k,v in all_docs_data.items():
    if k in relevant_docs:
        docs_data[k]=all_docs_data[k]
        
len(docs_data)

456

In [7]:
docs_data['39890c5f9109dcf8e892c4fc6eaa1fd6']

{'content': ['39890c5f9109dcf8e892c4fc6eaa1fd6.t1.1\tN.J.\tTITLE\t-\n',
  '39890c5f9109dcf8e892c4fc6eaa1fd6.t1.2\tState\tTITLE\t-\n',
  '39890c5f9109dcf8e892c4fc6eaa1fd6.t1.3\ttroopers\tTITLE\t-\n',
  '39890c5f9109dcf8e892c4fc6eaa1fd6.t1.4\tshoot\tTITLE\t-\n',
  '39890c5f9109dcf8e892c4fc6eaa1fd6.t1.5\tman\tTITLE\t-\n',
  '39890c5f9109dcf8e892c4fc6eaa1fd6.t1.6\t,\tTITLE\t-\n',
  '39890c5f9109dcf8e892c4fc6eaa1fd6.t1.7\t76\tTITLE\t-\n',
  '39890c5f9109dcf8e892c4fc6eaa1fd6.t1.8\t,\tTITLE\t-\n',
  '39890c5f9109dcf8e892c4fc6eaa1fd6.t1.9\tafter\tTITLE\t-\n',
  '39890c5f9109dcf8e892c4fc6eaa1fd6.t1.10\t911\tTITLE\t-\n',
  '39890c5f9109dcf8e892c4fc6eaa1fd6.t1.11\tcall\tTITLE\t-\n',
  '39890c5f9109dcf8e892c4fc6eaa1fd6.t1.12\tmix\tTITLE\t-\n',
  '39890c5f9109dcf8e892c4fc6eaa1fd6.t1.13\t-\tTITLE\t-\n',
  '39890c5f9109dcf8e892c4fc6eaa1fd6.t1.14\tup\tTITLE\t-\n',
  '39890c5f9109dcf8e892c4fc6eaa1fd6.b1.1\tAn\tBODY\t-\n',
  '39890c5f9109dcf8e892c4fc6eaa1fd6.b1.2\tinvestigation\tBODY\t-\n',
  '39890c5f9

In [8]:
with open(out_pickle, 'wb') as p:
    pickle.dump(docs_data, p)

### 2) Increase ambiguity by giving everyone the same first or last name

In [9]:
input_path='../data/input'
gva_input_annotations_file = '%s/annotation/participants_input.p' % input_path



#### 2a) Increase ambiguity by giving everyone the same first name

In [14]:
new_first_name = "John"

gva_samefirstname_annotations = '%s/annotation/participants_samefirstname.p' % input_path

with open(gva_input_annotations_file, 'rb') as f:
    gva_input_annotations=pickle.load(f)
    for doc_id, data in gva_input_annotations.items():
        for participant_id, participant_info in data.items():
            if 'Name' in participant_info:
                old_name=participant_info['Name']
                first_name, *other_names = old_name.split()
                participant_info['Name']=' '.join([new_first_name] + other_names)
                print(old_name, participant_info["Name"])
    with open(gva_samefirstname_annotations, 'wb') as w:
        pickle.dump(gva_input_annotations, w)

 Nathaniel Jones John Jones
 E.C. Robinson John Robinson
 Kendall Reed John Reed
 Will Harris John Harris
 Steven Coleman John Coleman
 Christopher Roupe John Roupe
 Christopher Roupe John Roupe
 Brady Osborne John Osborne
 Matt Anderson John Anderson
 Brady Osborne John Osborne
 Matt Anderson John Anderson
 Matt Anderson John Anderson
 Brady Osborne John Osborne
 Braison Howard John Howard
 Braison Howard John Howard
 Briar Newsome John Newsome
 Briar Newsome John Newsome
 Emmanuel Arrington John Arrington
 Sieyumbe Arrington John Arrington
 Cristobal Guerroro John Guerroro
 Anthony Heard John Heard
 Clarance Washington John Washington
 Olivia Washington John Washington
 Onya Washington John Washington
 Omesha Washington John Washington
 Shauna Washington John Washington
 Shauna Washington John Washington
 Omesha Washington John Washington
 Onya Washington John Washington
 Olivia Washington John Washington
 Clarance Washington John Washington
 Matthew Regula John Regula
 Kyle Carney J

#### 2b) Increase ambiguity by giving everyone the same last name

In [15]:
new_last_name = "Smith"

gva_samelastname_annotations = '%s/annotation/participants_samelastname.p' % input_path

with open(gva_input_annotations_file, 'rb') as f:
    gva_input_annotations=pickle.load(f)
    for doc_id, data in gva_input_annotations.items():
        for participant_id, participant_info in data.items():
            if 'Name' in participant_info:
                old_name=participant_info['Name']
                *other_names, last_name = old_name.split()
                participant_info['Name']=' '.join(other_names + [new_last_name])
                print(old_name, participant_info["Name"])
    with open(gva_samelastname_annotations, 'wb') as w:
        pickle.dump(gva_input_annotations, w)

 Nathaniel Jones Nathaniel Smith
 E.C. Robinson E.C. Smith
 Kendall Reed Kendall Smith
 Will Harris Will Smith
 Steven Coleman Steven Smith
 Christopher Roupe Christopher Smith
 Christopher Roupe Christopher Smith
 Brady Osborne Brady Smith
 Matt Anderson Matt Smith
 Brady Osborne Brady Smith
 Matt Anderson Matt Smith
 Matt Anderson Matt Smith
 Brady Osborne Brady Smith
 Braison Howard Braison Smith
 Braison Howard Braison Smith
 Briar Newsome Briar Smith
 Briar Newsome Briar Smith
 Emmanuel Arrington Emmanuel Smith
 Sieyumbe Arrington Sieyumbe Smith
 Cristobal Guerroro Cristobal Smith
 Anthony Heard Anthony Smith
 Clarance Washington Clarance Smith
 Olivia Washington Olivia Smith
 Onya Washington Onya Smith
 Omesha Washington Omesha Smith
 Shauna Washington Shauna Smith
 Shauna Washington Shauna Smith
 Omesha Washington Omesha Smith
 Onya Washington Onya Smith
 Olivia Washington Olivia Smith
 Clarance Washington Clarance Smith
 Matthew Regula Matthew Smith
 Kyle Carney Kyle Smith
 Mat

 Sarah Bonner Sarah Smith
 Hunter Richardson Hunter Smith
 Jaylen Pender Jaylen Smith
 Herbert Harr Herbert Smith
 Herbert Harr Herbert Smith
 Thomas Guitierrez Thomas Smith
 Daylee Newman Daylee Smith
 Carina Mancera Carina Smith
 Jennabel Anaya Jennabel Smith
 Carina Mancera Carina Smith
 Jennabel Anaya Jennabel Smith
 Brandon Colbert Brandon Smith
 Michael Parsons Michael Smith
 Michael Parsons Michael Smith
 Thomas Meisenheimer Thomas Smith
 Walter Meisenheimer Walter Smith
 Ibram Hanna Ibram Smith
 Ibram Hanna Ibram Smith
 Thomas Lane Thomas Smith
 Cade Hartwig Cade Smith
 Emiko Diaz Emiko Smith
 Jesus Diaz Jesus Smith
 Jayden Norton Jayden Smith
 Jada Page Jada Smith
 Dakota Schriefer Dakota Smith
 Treyvon Tyson Treyvon Smith
 Coleton Weatherford Coleton Smith
 Blake Shurling Blake Smith
 Coleton Weatherford Coleton Smith
 Blake Shurling Blake Smith
 Wakeel Stewart Wakeel Smith
 Maurice Bellamy Maurice Smith
 Tyree Johnson Tyree Smith
 Darryl Freeman Darryl Smith
 Jacob Setter Ja

### 3. Transform gold data to JSON with cluster IDs

In [7]:
which_partition='partial'
gold_file = '../data/gold/%s/participants_gold.p' % which_partition
out_gold_file = '../data/gold/%s/participants.json' % which_partition

with open(gold_file, 'rb') as g:
    gold_data = pickle.load(g)

In [8]:
gold_json=utils.transform_gold_to_json(gold_data)

In [9]:
with open(out_gold_file, 'w') as w:
    json.dump(gold_json, w)